In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,Dense,Dropout
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

from tensorflow.keras.optimizers import Adam

from gensim.models import Word2Vec

In [3]:
from google.cloud import storage
import os
import io
client = storage.Client()
bucket = client.get_bucket('nlp_final_data')

blob = bucket.blob('top_songs_final.csv')
content = blob.download_as_string()

df = pd.read_csv(io.BytesIO(content))

In [5]:
df.columns

Index(['Unnamed: 0', 'title', 'tag', 'artist', 'year', 'views', 'features',
       'lyrics', 'id', 'language_cld3', 'language_ft', 'language',
       'lyrics_cleaned', 'data_tokenized', 'num_tokens'],
      dtype='object')

In [7]:
# # separate the dataset into the target variable and features
X = df['data_tokenized']
y = df['tag']

In [8]:
# convert the lyrics to numerical vectors using bag-of-words representation
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X_vec = vectorizer.fit_transform(X)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

# we will need this later
num_words = len(tokenizer.word_index)+1
X_tok = tokenizer.texts_to_sequences(X)

max_log_length = 500
X_processed = pad_sequences(X_tok, maxlen=max_log_length)
# # convert the sparse matrix to a numpy array
# X_balanced_vec = X_balanced_vec.toarray()

In [9]:
# split the balanced data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2,stratify=y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2,stratify=y, random_state=42)

In [10]:
# Encode the class labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert the integer encoded labels to one-hot encoded vectors
num_classes = len(label_encoder.classes_)
y_train_onehot = np_utils.to_categorical(y_train_encoded, num_classes)
y_test_onehot = np_utils.to_categorical(y_test_encoded, num_classes)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# Build and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

     country       0.32      0.05      0.09      9312
         pop       0.24      0.02      0.03      9360
         rap       0.47      0.74      0.58      9035
          rb       0.29      0.01      0.01      9375
        rock       0.24      0.79      0.37      9284

    accuracy                           0.32     46366
   macro avg       0.31      0.32      0.22     46366
weighted avg       0.31      0.32      0.21     46366

